In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd
from collections import Counter
import itertools

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# CSVの読み込み
wikiData = pd.read_csv("csv/train.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場（みなみがおかぼくじょう）は、栃木県那須高原に本拠を置く...,1
2,岩瀬牧場 岩瀬牧場（いわせぼくじょう）は、日本の福島県岩瀬郡鏡石町にある牧場である。 ...,1
3,"クラギ クラギ株式会社（くらぎ、""KURAGI Co.Ltd."" ）は、三重県を中心に「...",1
4,"ハッピーネモファーム 株式会社ハッピーネモファーム（""""）は、北海道浦河郡浦河町にある競...",1


In [3]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger( "-Ochasen" )
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] == "名詞":
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [4]:
replaceList = ['(',')','"']
cleanedData = wikiData.replace(replaceList,'')

In [5]:
cleanedData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場（みなみがおかぼくじょう）は、栃木県那須高原に本拠を置く...,1
2,岩瀬牧場 岩瀬牧場（いわせぼくじょう）は、日本の福島県岩瀬郡鏡石町にある牧場である。 ...,1
3,"クラギ クラギ株式会社（くらぎ、""KURAGI Co.Ltd."" ）は、三重県を中心に「...",1
4,"ハッピーネモファーム 株式会社ハッピーネモファーム（""""）は、北海道浦河郡浦河町にある競...",1


In [6]:
# Numpy配列へ変換
X = cleanedData.text.values
y = cleanedData.target.values

In [7]:
words = []
for text in X:
    words.append(tokenize(text))

In [8]:
words_1d = itertools.chain.from_iterable(words)

In [9]:
counter = Counter(words_1d)
for word, count in counter.most_common():
    print(f"{word}: {count}")

年: 78766
月: 37518
株式会社: 23159
日: 21069
1: 17048
市: 16707
県: 15887
事業: 15870
こと: 14686
会社: 14427
3: 12235
2: 12121
販売: 12089
店: 11783
店舗: 11498
本社: 10966
日本: 10848
平成: 9956
企業: 9937
4: 9361
製造: 8702
東京: 8541
現在: 7968
バス: 7822
グループ: 7811
ため: 7425
者: 7123
設立: 7065
10: 6761
区: 6674
開発: 6531
化: 6406
5: 6349
円: 6040
展開: 5959
昭和: 5903
的: 5838
子会社: 5706
放送: 5648
メーカー: 5428
営業: 5403
後: 5360
社: 5343
7: 5311
運営: 5237
6: 5206
経営: 5182
所: 4918
用: 4913
9: 4837
8: 4707
大阪: 4595
ブランド: 4570
12: 4536
変更: 4525
開始: 4508
製品: 4502
駅: 4501
創業: 4437
商品: 4351
等: 4319
中心: 4316
11: 4180
同社: 4134
サービス: 4122
社名: 3967
都: 3936
町: 3850
路線: 3798
もの: 3766
合併: 3753
使用: 3740
工場: 3708
中: 3609
名: 3545
億: 3543
よう: 3483
業務: 3458
これ: 3439
同年: 3433
その後: 3399
一部: 3231
向け: 3195
人: 3190
運行: 3163
社長: 3149
時: 3105
線: 3046
関連: 3035
部: 3034
当時: 3022
一: 3010
車: 2989
機器: 2981
万: 2967
生産: 2956
機: 2951
）: 2933
の: 2918
主: 2896
番組: 2884
発表: 2861
位: 2855
株式: 2784
ほか: 2757
内: 2731
利用: 2722
2008: 2709
自動車: 2701
他: 2668
食品: 2652
品: 2650
システム: 

ライブ: 261
本拠地: 261
特急: 261
模型: 261
知名度: 260
会議: 260
箇所: 260
タイトル: 260
36: 260
SC: 260
55: 259
無関係: 259
八: 259
5000: 259
差: 259
表明: 259
投入: 259
代替: 259
準備: 259
マーケティング: 259
貨物: 259
方向: 259
国外: 258
自治体: 258
計測: 258
黒字: 257
興業: 257
マネー: 257
1974: 256
機材: 256
解体: 256
合成: 256
民間: 255
困難: 255
建て: 255
ジャスコ: 255
全域: 255
今: 255
変化: 255
ファミリー: 255
持分: 254
国産: 254
うどん: 254
工作: 254
日本テレビ: 253
ホームページ: 253
統括: 253
曲: 252
規制: 252
改組: 251
財閥: 251
登記: 251
公表: 251
ATM: 251
非常: 250
同月: 250
命令: 250
吉本: 250
比: 249
ソリューション: 249
1978: 248
子供: 248
復活: 248
44: 248
中小: 248
方: 248
テレビ朝日: 248
配置: 248
きっかけ: 248
フリー: 247
前半: 247
航空機: 247
大震災: 246
1963: 246
公園: 246
商会: 246
山田: 246
段階: 246
日立製作所: 246
樹脂: 246
車庫: 245
ドラマ: 245
承認: 245
朝: 244
注目: 244
前後: 244
39: 244
38: 244
三和: 244
1968: 243
1961: 243
ランド: 243
47: 243
水戸: 243
いすゞ: 243
注力: 242
佐藤: 242
品川: 242
外部: 242
ブルー: 242
編集: 242
上海: 241
松山: 241
1976: 241
フェリー: 241
話題: 240
春: 240
満了: 240
靴: 240
新車: 240
連合: 239
事務: 239
北部: 239
保守: 239
決済: 239
伊勢丹: 239
デバイス: 239
管内: 238

要望: 140
収録: 140
事例: 140
調理: 140
役割: 140
藤沢: 140
化成: 140
敗退: 140
苫小牧: 139
出力: 139
拡充: 139
だい: 139
惣菜: 139
味噌: 139
番目: 139
正規: 139
生命: 139
ハイビジョン: 139
売店: 138
返済: 138
配給: 138
物件: 138
午前: 138
注意: 138
毎月: 138
熱: 138
優良: 138
人口: 138
AC: 138
関与: 138
現代: 138
プレス: 138
見学: 138
美容: 138
略: 138
不明: 138
e: 138
オート: 138
側面: 137
創刊: 137
ニュー: 137
以内: 137
復興: 137
豊富: 137
スキー: 137
読売: 137
武田: 137
太郎: 137
成形: 137
ほう: 137
西武百貨店: 137
紡績: 137
原子力: 136
アナウンス: 136
ピーク: 136
ホームスタジアム: 136
中島: 136
洋: 136
岩田屋: 136
ドライバー: 136
ホンダ: 136
ハイヤー: 136
西武鉄道: 136
お客様: 135
アンテナ: 135
宇部: 135
ジャスダック: 135
順調: 135
徹底: 135
現行: 135
信頼: 135
午後: 135
75: 135
加藤: 135
コンピューター: 135
アーケード: 135
亭: 134
格: 134
日間: 134
創造: 134
オーストラリア: 134
バスケットボール: 134
三洋電機: 134
CI: 134
１: 134
信号: 134
あさひ: 134
終戦: 134
藤: 134
キッチン: 134
キャピタル: 134
個別: 134
AT: 134
難: 133
コラボレーション: 133
業種: 133
1943: 133
ヘルスケア: 133
著作: 133
サイド: 133
常務: 133
豆: 133
刊行: 133
来店: 133
アプリケーション: 133
日産ディーゼル: 133
飼料: 132
社団: 132
認知: 132
言葉: 132
違い: 132
末期: 132
鳥栖: 132
装備: 132
ボーイング: 13

生放送: 88
科: 88
横須賀: 88
手段: 88
感謝: 88
日曜日: 88
ウイスキー: 88
象徴: 88
藤田: 88
1933: 88
スタンダード: 88
プラモデル: 88
クリスタル: 88
翔: 88
厚木: 88
センサ: 88
Wi: 88
加算: 88
行き: 88
八千代: 87
市長: 87
芝: 87
兵衛: 87
中学校: 87
ウエスト: 87
メイド: 87
コンサート: 87
徳: 87
人々: 87
兼任: 87
簡易: 87
自宅: 87
冠: 87
中旬: 87
弘前: 87
親: 87
羽田: 87
粉: 87
予報: 87
草津: 87
不可能: 87
刷新: 87
ヘッド: 87
展: 87
SA: 87
早朝: 87
先述: 87
授業: 87
為替: 87
講座: 87
ゼネラル: 87
松下電器産業: 87
バイク: 87
金利: 87
ワンステップ: 87
ウエルシア: 87
幅: 86
安心: 86
トップクラス: 86
辰: 86
小牧: 86
リフォーム: 86
早期: 86
美術館: 86
セールス: 86
清掃: 86
着工: 86
文京: 86
三井物産: 86
坂: 86
地下街: 86
ベーカリー: 86
均一: 86
三共: 86
買い付け: 86
製粉: 86
昼: 86
進: 86
佐世保: 86
月頃: 86
エンブレム: 86
発車: 86
緩和: 86
アイ・ホールディングス: 86
明石: 86
SUPER: 86
岡: 86
起点: 86
ストアー: 86
ベイ: 86
車輌: 86
1932: 85
娘: 85
メリット: 85
86: 85
住友銀行: 85
努力: 85
指揮: 85
三浦: 85
主軸: 85
評判: 85
谷: 85
例外: 85
初め: 85
華: 85
デザイナー: 85
夕方: 85
1931: 85
元気: 85
実証: 85
新人: 85
鋼材: 85
ルノー: 85
互換: 85
エイト: 85
別館: 85
発見: 84
大垣: 84
考え: 84
依存: 84
子ども: 84
教授: 84
英国: 84
風力: 84
京セラ: 84
定時: 84
接着: 84
研: 84
給食: 84
文庫: 84
路面: 84
独特: 84


ル・マン: 49
コイン: 49
防水: 49
プリンタ: 49
無効: 49
能: 49
GP: 49
コントロール: 49
中学生: 49
ゼロックス: 49
荷物: 49
垂直: 49
両備: 49
駿: 49
ルネサス: 49
UQ: 49
通話: 49
阪神電気鉄道: 49
EV: 49
佑: 49
新京成電鉄: 49
居抜き: 49
松竹: 49
レンタルビデオ: 49
ホット: 48
統廃合: 48
値段: 48
福利: 48
繋がり: 48
屈指: 48
TOTO: 48
コンデンサ: 48
陶磁器: 48
北海: 48
近畿日本鉄道: 48
雰囲気: 48
罰金: 48
ミサワホーム: 48
モットー: 48
知多: 48
東海道新幹線: 48
幸せ: 48
一緒: 48
転籍: 48
相鉄: 48
テロップ: 48
上部: 48
負傷: 48
社是: 48
de: 48
流用: 48
83: 48
多彩: 48
藤井: 48
貯蔵: 48
ガ: 48
和泉: 48
挿入: 48
相模: 48
すき: 48
コカ・コーラボトリング: 48
通年: 48
高津: 48
三条: 48
めし: 48
たて: 48
黄: 48
明日: 48
マクドナルド: 48
マック: 48
通商: 48
ふるさと: 48
名寄: 48
近接: 48
連載: 48
大野: 48
クラウン: 48
母: 48
放射線: 48
ディスカウントストア: 48
プレミア: 48
打診: 48
2500: 48
階段: 48
パリ: 48
敬: 48
おおさか: 48
服部: 48
トレード: 48
787: 48
民生: 48
頓挫: 48
中学: 48
接収: 48
研削: 48
ロク: 48
生体: 48
ポート: 48
FDK: 48
蓄電池: 48
遊技: 48
トミー: 48
泊: 48
ミドリ: 48
プランタン: 48
西濃運輸: 48
知床: 48
バード: 47
池: 47
フェア: 47
プレゼント: 47
園芸: 47
たい: 47
標榜: 47
恐れ: 47
なん: 47
良好: 47
屋外: 47
西新宿: 47
東海旅客鉄道: 47
女: 47
衆議院: 47
０: 47
播磨: 47
とうかい: 47
身: 47
360: 47
新居浜: 47
決

...

妖星: 1
ゴラス: 1
マタンゴ: 1
ホラ: 1
ドゴラ: 1
サンダ: 1
ガイラ: 1
ゴジラ・エビラ・モスラ: 1
ギャオス: 1
ゴジラ・ミニラ・ガバラ: 1
望郷: 1
恋歌: 1
ヘドラ: 1
ガイガン: 1
忘れな草: 1
年少: 1
蹉跌: 1
竜馬: 1
金環: 1
蝕: 1
相合傘: 1
ＶＳ: 1
野麦峠: 1
チャールストン: 1
怪異: 1
伽耶: 1
ビューティフル・ドリーマー: 1
SU: 1
郷愁: 1
火垂る: 1
ルビィ: 1
ビオランテ: 1
四万十川: 1
哀しみ: 1
KAMIKAZE: 1
キッズ・リターン: 1
極道: 1
夜曲: 1
ＲＥＶＥＮＧＥ: 1
キュア: 1
ナージャ: 1
ＴＨＥ: 1
ＡＵＲＯＲＡ: 1
Dolls: 1
ドールズ: 1
阿修羅: 1
娘道成寺: 1
パッチギ: 1
ローレライ: 1
輪廻: 1
ばあちゃん: 1
流刑: 1
劔: 1
門外: 1
ゴールデンスランバー: 1
DOG×POLICE: 1
兵団: 1
ビヨンド: 1
桐島: 1
旅立ち: 1
！～: 1
なあなあ: 1
蜩: 1
子分: 1
バカヴォン: 1
フランダース: 1
星ガ丘: 1
チアダンス: 1
カチコチ: 1
爆盛: 1
カンフーボーイズ: 1
大乱: 1
フォルトゥナ: 1
月夜: 1
サンライズ・スタジオコメット・: 1
ヴォルカン: 1
volcan: 1
就行: 1
KAPS: 1
Arcadia: 1
アマミノクロウサギ: 1
西神田: 1
他校: 1
ゅうだいしんがく: 1
ソンポケア: 1
第一エージェンシー: 1
MRS: 1
レンテム: 1
遍歴: 1
13349: 1
カンター: 1
cantare: 1
aller: 1
富山湾: 1
ニホンカモシカ: 1
リーグスタディグループ: 1
聡司: 1
73205: 1
4306: 1
貴義: 1
連係: 1
13639: 1
4474: 1
カスタムズ: 1
進退伺: 1
ヤン・ヘジュン: 1
YFCMD: 1
キム・ヨングン: 1
カンボジア・カンボジアンタイガー: 1
チエンマイ: 1
脇本: 1
イカザ: 1
グレミオ・オザスコ・アウダックス: 1
貴博: 1
マドゥレイラ: 1
ルーカス・ダウベルマン: 1
龍治: 1
戸根: 1

ブレーダ: 1
ヴィヴィ: 1
ナショナルトレセンコーチ: 1
貝崎: 1
古部: 1
パジャルド: 1
レアル・ムルシア: 1
ソン・ヨンミン: 1
ヨルディ・バイス: 1
ハイロ・モリージャス: 1
大知: 1
髙田: 1
実数: 1
空き家: 1
多良見: 1
⼟地: 1
⽤: 1
観覧席: 1
⽤地: 1
NAGASAKI: 1
ょうえいこうぎょう: 1
堂山: 1
”（: 1
オフィシャルプロバイダー: 1
BrandAsset: 1
ストラテジックプランニング: 1
クリエーティブ: 1
ボルボ・カーズ・ジャパン: 1
安全ピン: 1
PAIN: 1
GAIN: 1
プロモライオン: 1
口紅: 1
ディーワイアール: 1
サドラー・アンド・ヘネシー: 1
バーソン・マーステラ: 1
Labstore: 1
Ceremony: 1
サンセルモ: 1
ブレスユー: 1
タレントエージェンシー・ブレスユー: 1
Bless: 1
クリエイティブスタジオ・ブレスユー: 1
ワタナベエンターテインメントグループ: 1
LABO: 1
PROSHOP: 1
カーライル・ジャパン・パートナーズ: 1
プライベートエクイティファンド: 1
メティス: 1
OSSHY: 1
アヤ子: 1
firstpromotion: 1
アンダース・ルンドクヴィスト: 1
吉里: 1
ソルボンヌ大学: 1
TGM: 1
AAM: 1
Diplomat: 1
LEVEL: 1
レベルチェックテスト: 1
ボキャブラリー: 1
行政府: 1
最高裁判所: 1
クーリングオフ: 1
近鉄難波: 1
1803: 1
4415: 1
山原: 1
催促: 1
未納: 1
典明: 1
昨春: 1
5485: 1
Bermuda: 1
別紙: 1
RICH: 1
PENINSULA: 1
PROFITS: 1
ベルリッツ: 1
徳山大学: 1
浜山: 1
県勢: 1
金村: 1
賢志: 1
ViewStyles: 1
ニフス: 1
Fitness: 1
HOYO: 1
Atletico: 1
ELAN: 1
オフィシャルカフェ: 1
八ケ岳: 1
ザイラーバレースキー: 1
シーアール・エス: 1
川鉄商事: 1
Krei: 1
ranch: 1
nest: 1
チケットキング: 1
昼過ぎ: 1


トラロックエンターテインメント: 1
エムアンドエーキャピタルパートナーズ: 1
アドバイザリー・: 1
6080: 1
座元: 1
卑賤: 1
勝五郎: 1
座が: 1
掣肘: 1
團十郎: 1
桧: 1
副官: 1
フォービアン・バワーズ: 1
年忘れ: 1
定紋: 1
づくし: 1
草案: 1
武中: 1
閉場: 1
河竹: 1
黙阿弥: 1
ギンザ: 1
カブキザ: 1
立見: 1
乃内: 1
ぜいたく: 1
休演: 1
弥次喜多: 1
隠密: 1
斬り捨て御免: 1
狂四郎: 1
無頼: 1
井崎: 1
seeds: 1
ASOBISYSTEM: 1
ぱみゅぱみゅが: 1
イベンター: 1
ぱみゅぱみゅの: 1
コンサバファッション: 1
アミューズメントメイドカジノ: 1
クラップス: 1
ドボン: 1
オンラインポーカーサイト: 1
PokerStars: 1
Paradise: 1
Casino: 1
Walkerhill: 1
七隈: 1
11400: 1
南欧: 1
ファイナンスアドバイザリー: 1
デロイトトーマツグループ: 1
エグゼキューション: 1
ブレイン・アンド・キャピタル・ホールディングス: 1
プライスウォーターハウスクーパーズ: 1
高階: 1
澄人: 1
Lightning: 1
雷鳴: 1
チアダンススクール: 1
広陵: 1
広島経済大学: 1
敏光: 1
ジョー・ナバーロ: 1
トライアンズアンバサダー: 1
常宿: 1
名刹: 1
對旭: 1
道幅: 1
仁王門: 1
師田: 1
数寄屋造: 1
洋才: 1
希典: 1
HEIGORO: 1
ORAHO: 1
ょこう: 1
Meihan: 1
kintetsu: 1
カッコーツアー: 1
アーツビジョン: 1
夏海: 1
ソーシャルスタートアップ: 1
303354: 1
ポストコンサル: 1
ビズリーチ: 1
CEG: 1
ょういくしすてむ: 1
建一: 1
東野田: 1
テレビデオ: 1
Lpa: 1
祖師谷: 1
宮子: 1
ララガーデン: 1
鶴巻温泉: 1
相武台: 1
ペガサート: 1
サントムーン: 1
柿田: 1
マーゴ: 1
アクアウォーク: 1
マーサ: 1
尾平: 1
インターネットテレビサイト: 1
Jibannet: 1
6072: 1
姪浜:

龍平: 1
ハン・ヨンテ: 1
ユースアカデミーサッカースクール: 1
アルフット: 1
エス・エス: 1
ジェイ・エス・エス: 1
牛込: 1
スペランツァ: 1
ノーヴァ: 1
レベルファイブスタジアム: 1
春日公園: 1
湯屋: 1
メルモンテ: 1
プリツカー: 1
ヴェンチューリ: 1
エクステリアデザイン: 1
マルセル・ブロイヤー: 1
ワシリーチェア: 1
ミース・ファン・デル・ローエ: 1
ブルーノチェア: 1
BCJ: 1
エヒメスポーツエンターテインメント: 1
水軍: 1
オレンジヴィーナス: 1
閻魔: 1
デビル: 1
メジロ: 1
ディーディー: 1
デビルガールズ: 1
コンパルホール: 1
コミュセン: 1
ホプキンス: 1
ジョワン・オールドハム: 1
プレイオフ・: 1
ローレンス: 1
3918: 1
プレイングアシスタントコーチ: 1
忠晴: 1
セネガル: 1
レプリカジャージ: 1
ブライアン・ロウサム: 1
ラシャード・シングルトン: 1
魅了: 1
エル・ジェイ・ヘップ: 1
トニー・ハンソン: 1
ロティック: 1
ヘップ: 1
テンポラリーゲームオペレーション: 1
バーデル・ジョーンズ: 1
大城: 1
デヴォーン・ジャイルズ: 1
コンパルホール・: 1
シーズンアウェイ: 1
ケービーシートータルサービス: 1
サイモン: 1
エヒメスポーツエンターテイメント: 1
ジョシュア: 1
ウィリアムス・ジュニア: 1
リチャード・グレスマン: 1
冨岡: 1
コナー: 1
矢代: 1
真由: 1
雑踏: 1
コアズオートサービス: 1
コアズメンテ: 1
ホームセキュリティーシステム: 1
コアズネット: 1
常滑競艇: 1
修造: 1
池村: 1
西社: 1
セノン: 1
ょういくこうしゃ: 1
マンツー: 1
エデュネット: 1
スクールツアーシップ: 1
暗示: 1
リソーグループ: 1
1477: 1
天坊: 1
認諾: 1
ハーフコートマンツーマン: 1
オフィシャルアーティスト: 1
モーターサイクルレーシング: 1
ギガキャッツ: 1
ボッシュブルーウィンズ: 1
フェニックスコミュニケーションズ: 1
ジョシュ・ペッパーズ: 1
アダム・ザーン: 1
ディナモ・モスクワ: 1
チャンピオンシッ

Maryland: 1
トランスレーショナル: 1
Harvard: 1
グリーンピア: 1
メディポリスエネジー: 1
叡: 1
伺: 1
院生: 1
御倉: 1
アットワン: 1
アヤーズカフェ: 1
ELF: 1
agency: 1
ヴァーナル: 1
パン・マドレーヌ・ピット: 1
エンゲージリング: 1
Plume: 1
cheri: 1
プリューム: 1
ブランドラ・パルレ: 1
PARLER: 1
ビューティーパートナーズ: 1
党員: 1
ざわこ: 1
戎本: 1
三重野: 1
有子: 1
内舘: 1
牧子: 1
横内: 1
パク・トンハ: 1
良知: 1
立科: 1
メダルゲーム・クレーンゲーム: 1
スノーパーク: 1
夏山: 1
プリキュアルーム: 1
藤城: 1
ポタ: 1
ポタキッズクラブ: 1
TIAMO: 1
エフシーティアモ: 1
ゴールデンエイジ: 1
大進: 1
アイビージェー: 1
合コン: 1
びしじどうしゃ: 1
MAE: 1
ASSiS: 1
日産コーエー: 1
サンガー: 1
アイチ・エステート・サービス: 1
ジヤトコサービス: 1
ジヤトコフード: 1
キュービーネットホールディングス: 1
BARBER: 1
ATHLETIC: 1
荒川沖: 1
ポセイドン: 1
ライクキッズネクスト: 1
ライクケアネクスト: 1
Wedge: 1
グロース: 1
Innovator: 1
ウェッジホールディングスグループ: 1
エリアフランチャイザー・メガフランチャイジー: 1
ローソンアーバンワークス: 1
Apan: 1
エーパンスリーセブン: 1
ゼント: 1
ベルトラン・ガショー: 1
ラルースチーム: 1
ラルース: 1
一ツ山: 1
トヨタ・レーシング: 1
ゼントスイーティーズ: 1
class: 1
ユーラシア: 1
ょこうしゃ: 1
熟年: 1
探訪: 1
IREP: 1
デジタルマーケティングエージェンシー: 1
宮里: 1
早苗: 1
Tempstaff: 1
テンプスタッフ・ウィッシュ: 1
テンプスタッフプラス: 1
テンブロス: 1
オリコミ: 1
づり: 1
サーキュレーション: 1
DAISHINTO: 1
レイディアントシティ: 1
ル・グランブルー: 1
金沢文庫: 1
バスエルガ・: 1
バ

In [10]:
textValues, counts = zip(*counter.most_common(10))

In [11]:
cleanedData = cleanedData.replace(textValues,'')

In [12]:
# CSV出力
cleanedData.to_csv("csv/train_cleaned.csv",index = False, encoding='utf-8')